<h3 style="text-align: center;"> Language Models Lab 1</h3>
<h5 style="text-align: center;"> Wenjie Hu 343312</h5>



### P1

In [1]:
import torch
import itertools
from itertools import product
from transformers import AutoModelForCausalLM, AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel

In [3]:
model_name = "gpt2"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.config.pad_token_id = model.config.eos_token_id if model.config.eos_token_id is not None else 50256

system_prompt = """
Introduction:
The user is contacting the assistant. The assistant is helpful and responds shortly.
Dialogue:"""

print()
while (user_prompt := input().strip()):

    prompt = f"""{system_prompt}
        User: \"{user_prompt}\"
        Assistant: \""""

    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

    generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=100,
      do_sample=True,
      penalty_alpha=0.6,
      top_k=10,
      pad_token_id=model.config.pad_token_id
    )

    output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(30 * '-', flush=True)
    print(f"User: \"{user_prompt}\"", flush=True)
    response = output.removeprefix(prompt).strip().replace("\\n", "\n").split("\n")[0].split("\"")[0] + "\""
    system_prompt = prompt + response
    print(f"GPT: \"{response}", flush=True)



hello
------------------------------
User: "hello"
GPT: "hi"
what is your favoriate  color
------------------------------
User: "what is your favoriate  color"
GPT: "you know "
I don't know
------------------------------
User: "I don't know"
GPT: "you are so nice"
how to good morning in polish
------------------------------
User: "how to good morning in polish"
GPT: "how to good breakfast in polish"
how is the weather like today
------------------------------
User: "how is the weather like today"
GPT: "why don't you just come over here and talk to me "


KeyboardInterrupt: Interrupted by user

### P2

In [4]:

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

def score_sentence(sentence, model, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss.item()
    return loss

def sentence_prob(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss.item()
    return -loss

def format_sentence(words):
    sentence = ' '.join(words)
    return sentence.capitalize() + "."

def generate_permutations(words):
    permutations = list(itertools.permutations(words))
    scored_sentences = []
    for perm in permutations:
        sentence = format_sentence(perm)
        score = score_sentence(sentence, model, tokenizer)
        scored_sentences.append((sentence, score))
    scored_sentences.sort(key=lambda x: x[1])
    return scored_sentences

In [5]:
words = "John likes spinach very much".split()
# words = "Squirrels live in the park".split()
print("=== All Permutations ===")
permutations = generate_permutations(words)
for sentence, score in permutations[:10]:
    print(f"{sentence} (Score: {score})")

=== All Permutations ===
John very much likes spinach. (Score: 5.476344108581543)
John likes spinach very much. (Score: 5.8143510818481445)
John likes very much spinach. (Score: 5.995521545410156)
Spinach very much likes john. (Score: 6.055186748504639)
Spinach likes john very much. (Score: 6.245360374450684)
Spinach john likes very much. (Score: 6.41658353805542)
Spinach likes very much john. (Score: 6.430897235870361)
Spinach john very much likes. (Score: 6.470345973968506)
John spinach likes very much. (Score: 7.153367519378662)
Spinach very much john likes. (Score: 7.340561389923096)


In [8]:

def sentence_prob(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss.item()
    return -loss

def test_synergy(w, v, wv):
    return sentence_prob(wv) > 0.85 * (sentence_prob(w) + sentence_prob(v))

def synergize(words):
    synergies = filter(lambda args: test_synergy(*args) and args[0] != args[1],
                       ((w, v, f"{w} {v}") for w, v in product(words, words)))

    return max(synergies, key=lambda args: sentence_prob(args[-1]), default=None)

def chunk_and_permute(words):
    words = set(words)
    while (s := synergize(words)):
        w, v, wv = s
        words.remove(w)
        words.remove(v)
        words.add(wv)  # Merge the best pair

    final_permutations = list(itertools.permutations(words))
    scored_permutations = []
    for perm in final_permutations:
        sentence = " ".join(perm)
        score = -sentence_prob(sentence)
        scored_permutations.append((sentence, score))

    scored_permutations.sort(key=lambda x: x[1], reverse=True)
    return scored_permutations[:10]


In [9]:
words = "Last night, I met a wonderful woman".split()
# words = "Last night, I met a wonderful woman who passionately talked about language model".split()
top_sentences = chunk_and_permute(words)
for sentence, score in top_sentences:
    print(f"{sentence} (Score: {score})")


a met wonderful night, I Last woman (Score: 9.791654586791992)
woman I Last a met wonderful night, (Score: 9.77700138092041)
a met wonderful night, I woman Last (Score: 9.74211311340332)
a met wonderful night, woman I Last (Score: 9.725557327270508)
I Last a met wonderful night, woman (Score: 9.626494407653809)
a met wonderful night, Last I woman (Score: 9.606175422668457)
a met woman Last I wonderful night, (Score: 9.561647415161133)
a woman met Last I wonderful night, (Score: 9.50749683380127)
a met Last I wonderful night, woman (Score: 9.440475463867188)
a woman Last I wonderful night, met (Score: 9.31971549987793)


## p3

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F

In [11]:
MODEL_NAME = 'flax-community/papuGaPT2'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/888k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [12]:
def cal_log_probs(logits: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
    log_probs = F.log_softmax(logits, dim=-1)
    label_log_probs = torch.gather(log_probs, 2, labels.unsqueeze(2)).squeeze(-1)
    return label_log_probs

def cal_accuracy(classifier_function) -> float:
    data = []
    with open("reviews_for_task3.txt", "r") as file:
        for line in file:
            label, *review_tokens = line.strip().split()
            review = " ".join(review_tokens)
            prediction = classifier_function(review)
            data.append(label == prediction)

    return sum(data) / len(data)

one approach is combine review with the label and get its log probability


In [13]:
def classify_review_1(review: str) -> str:
    labels = ["Opinia jest pozytywna.", "Opinia jest negatywna."]
    probs = {}

    for label in labels:
        input_text = f"{review} {label}"
        input_ids = tokenizer(input_text, return_tensors='pt')['input_ids'].to(DEVICE)

        with torch.no_grad():
            output = model(input_ids=input_ids)
            log_probs = cal_log_probs(output.logits[:, :-1, :], input_ids[:, 1:])
            probs[label] = torch.sum(log_probs).item()

    return "GOOD" if probs[labels[0]] > probs[labels[1]] else "BAD"

print(f"Classifier Accuracy: {cal_accuracy(classify_review_1) * 100:.2f}%")


Classifier Accuracy: 78.75%


another appoarch is classify based on the average log probability


In [14]:
# classify based on the average log probability
def classify_review_2(review: str, threshold: float = 0.5) -> str:

    input_ids = tokenizer(review, return_tensors='pt')['input_ids'].to(DEVICE)

    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = cal_log_probs(output.logits[:, :-1, :], input_ids[:, 1:])
        average_log_prob = torch.mean(log_probs).item()
    return "GOOD" if average_log_prob > threshold else "BAD"

print(f"Classifier Accuracy: { cal_accuracy(classify_review_2) * 100:.2f}%")


Classifier Accuracy: 50.00%


# p4

In [15]:
import torch
import os
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
MODEL_NAME = "eryk-mazus/polka-1.1b"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
INPUT_QUESTIONS_PATH = "task4_questions.txt"
OUTPUT_ANSWERS_PATH = "found_answers.txt"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)
model.generation_config.pad_token_id = tokenizer.pad_token_id
model.eval()

In [ ]:
def query(question, answer):
    """Format a single Q/A as part of the few-shot prompt."""
    return f"Pytanie: {question}\nOdpowiedź: {answer}"

EXAMPLES_DICT = {
    "Kiedy": [
        query("Kiedy wybuchła II wojna światowa?", "1 września 1939"),
        query("Kiedy zakończyła się epoka lodowcowa?", "około 10 tysięcy lat temu"),
        query("Kiedy wynaleziono druk?", "w XV wieku")
    ],
    "Ile": [
        query("Ile dni ma rok przestępny?", "366"),
        query("Ile członków liczy Unia Europejska?", "27"),
        query("Ile mamy podstawowych zmysłów?", "5")
    ],
    "Gdzie": [
        query("Gdzie znajduje się Wielki Kanion Kolorado?", "w stanie Arizona w USA"),
        query("Gdzie leży Machu Picchu?", "w Peru"),
        query("Gdzie można zobaczyć zorzę polarną?", "w pobliżu biegunów, m.in. w Skandynawii")
    ]
}

def get_examples_for_question(question):
    question_lower = question.lower()
    if question_lower.startswith("kiedy"):
        return EXAMPLES_DICT["Kiedy"]
    elif question_lower.startswith("gdzie"):
        return EXAMPLES_DICT["Gdzie"]
    elif question_lower.startswith("ile"):
        return EXAMPLES_DICT["Ile"]
    else:
        # Fallback: no specific few-shot examples
        return []

def ask(question, examples=None):
    """
    Generate an answer to the given 'question' using a few-shot prompt
    constructed from 'examples'.
    """
    if examples is None:
        examples = []
    system_prompt = "\n\n".join(examples)

    prompt = f"{system_prompt}\n\n{query(question, '')}"

    # Tokenize
    inputs = tokenizer([prompt], return_tensors="pt").to(DEVICE)
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=126,
        do_sample=True,
        penalty_alpha=0.6,
        top_k=10
    )

    output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # clean up
    response = (
        output.removeprefix(prompt)
              .strip()
              .replace("\\n", "\n")
              .split("\n")[0]
              .split("\"")[0]
    )
    return response

In [ ]:
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sentence_prob(sentence_txt):
    input_data = tokenizer(sentence_txt, return_tensors='pt')
    input_ids = input_data["input_ids"].to(DEVICE)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().item()

In [1]:
def main():
    if os.path.exists(OUTPUT_ANSWERS_PATH):
        os.remove(OUTPUT_ANSWERS_PATH)

    with open(INPUT_QUESTIONS_PATH, "r", encoding="utf-8") as fin, \
         open(OUTPUT_ANSWERS_PATH, "w", encoding="utf-8") as fout:

        for i, raw_question in enumerate(fin):
            question = raw_question.strip()
            # Print progress
            if i % 100 == 0:
                print(f"Processing question #{i}: {question}")
            examples_for_this_q = get_examples_for_question(question)
            answer = ask(question, examples_for_this_q)
            fout.write(answer + "\n")

    print("Done. Answers written to:", OUTPUT_ANSWERS_PATH)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Processing question #0: Która organizacja powstała wcześniej: Europejska Wspólnota Węgla i Stali czy Europejska Wspólnota Energii Atomowej?
Processing question #100: W którym województwie leży Toruń?
Processing question #200: Ile listów św. Piotra zawiera Nowy Testament?
Processing question #300: Jak z angielskiego nazywa się gęsta miejska mgła zawierająca cząstki spalin?
Processing question #400: W której dziedzinie polscy nobliści dostali najwięcej nagród?
Processing question #500: Jaka ciecz jest najczęściej mierzona w baryłkach?
Processing question #600: Który znak zodiaku przypada na okres 24.10–22.11?
Processing question #700: Jak nazywał się hetman kozacki, który wzniecił powstanie w 1648 r.?
Processing question #800: Przebiśniegi kwitną wiosną czy późną jesienią?
Processing question #900: Jak nazywa się wojsko walczące pieszo?
Processing question #1000: Jak nazywa się wzrost kursów papierów wartościowych lub cen towarów notowanych na giełdzie?
Processing question #1100: Jak naz

In [5]:
! python answer_check_for_task4.py


TOTAL SCORE: 42.0
